### Расчет тепловой схемы К-220-44

In [1]:
# Импортируем необходимые библиотеки
from CoolProp.CoolProp import PropsSI # параметры воды и пара
import numpy as np                    # работа с массивами
import pandas as pd                   # работа с таблицами
import sympy as sym                   # модуль для символьного исчисления

In [78]:
# Исходные данные Давления в МПа
P1 = 2.786 
P2 = 1.93 
P3 = 1.288 
P4 = 0.508 
P5 = 0.3 
P6 = 0.127 
P7 = 0.058 
P8 = 0.029
Pk = 0.005

Potb = np.array([P1, P2, P3, P4, P5, P6, P7, P8, Pk])

# Исходные данные по температурам, С
t1 = 229.8
t2 = 210.7
t3 = 196.2
t4 = 152.4
t5 = 134.4
t6 = 118
t7 = 101
t8 = 68.4

totb = np.array([t1, t2, t3, t4, t5, t6, t7, t8])

# Данные по КПД
KPDcvd = 0.79
KPDcnd = 0.865

# Начальные параметры пара
P0 = 4.4 * 10 **6
X0 = 1
tpv = 225

Ne = 220 
t0 = 450

### Определение энтальпий в отборах и построение процесса в h-S диаграмме 

In [80]:
# Определяем параметры в точке 0
h0 = np.round(PropsSI('H', 'P', P0, 'T', t0+273.15, 'IF97::Water') / 10**3, 3)

# Определяем параметры на входе в турбину
# Энтальпия на входе в турбине с учетом потерь

#S01 = np.round(PropsSI('S', 'P', P0*0.99,'H', h0*1000, 'IF97::Water') / 10**3, 3)   потом поменять

S01 = 6.907



# Давления пара в ЦВД
Pcvd = Potb[0 : 4]

# Определим теоретические энтальпии пара в ЦВД
hcvd_t = np.round(PropsSI('H','S', S01*1000, 'P', Pcvd*10**6, 'IF97::Water') / 10**3, 1)

# Определим действительные энтальпии пара в ЦВД
hcvd_i = np.round(h0 - (h0 - hcvd_t)*KPDcvd, 1)

# Определим энтальпию и температуру пара на выходе из СПП
tпп = 234+273.15
# Определим давление пара на входе в ЦНД 
Pcnd = P4 * 0.93
# Определим энтальпию и энтропию пара на входе в ЦНД
hcnd0 = np.round(PropsSI('H', 'P', Pcnd * 10 **6,'T', 234+273.15, 'IF97::Water') / 10**3, 3)
Scnd0 = np.round(PropsSI('S', 'P', Pcnd * 10 **6,'T', 234+273.15, 'IF97::Water') / 10**3, 3)
# Давления пара в ЦНД
Pcnd = Potb[4 : 9]
# Определим теоретические энтальпии пара в ЦНД
hcnd_t = np.round(PropsSI('H','S', Scnd0*1000, 'P', Pcnd*10**6, 'IF97::Water') / 10**3, 1)
# Определим действительные энтальпии пара в ЦВД
hcnd_i = np.round(hcnd0 - (hcnd0 - hcnd_t)*KPDcnd, 1)
# Энтальпии действительных отборов
hi_d = np.hstack((hcvd_i, hcnd_i))
# Создаем датафрейм с результатами и выводим в табличном виде
dat = {'Давления в отборах' : Potb,
    'Энтальпия в отборах теорет' : np.hstack((hcvd_t, hcnd_t)),
    'Энтальпия в отборах действит' : hi_d}

df_n = pd.DataFrame(dat)
df_n

,Давления в отборах,Энтальпия в отборах теорет,Энтальпия в отборах действит
0,2.786,3199.0,3225.6
1,1.930,3096.5,3144.6
2,1.288,2993.0,3062.8
3,0.508,2788.6,2901.3
4,0.300,2830.1,2843.4
5,0.127,2669.4,2704.4
6,0.058,2541.6,2593.9
7,0.029,2437.8,2504.1
8,0.005,2206.4,2303.9


In [81]:
S01

6.907

In [82]:
Scnd0

7.236

In [83]:
h0

3325.444

In [84]:
# Определим действительный теплоперепад турбины
Hi = h0 - hcvd_i[3] + hcnd0 - hcnd_i[4]

Hi

1049.0929999999998

In [85]:
hcvd_i[3]

2901.3

### Определим параметры воды в тракте основного конденсата

In [86]:
# Давление пара в подогревателях с учетом потерь в 5%
Pheater = 0.95 * (Potb[0 : 8])
# Определим температуры насыщения в подогревателях
ts_h = np.round(PropsSI('T', 'P', Pheater * 10 **6, 'Q', 0, 'IF97::Water') - 273.15, 1)
ts_k = np.round(PropsSI('T', 'P', Pk * 10 **6, 'Q', 0, 'IF97::Water') - 273.15, 1)
# Определяем энтальпию в конденсаторе
hdr_k = np.round(PropsSI('H', 'P', Pk * 10 **6, 'Q', 0, 'IF97::Water') / 10**3, 3)
# Добавим к Диапазону давлений давление в конденсаторе
Pheater1 = np.hstack((Pheater, Pk))
# Добавим к диапазону температур температура в конденсаторе
ts_h1 = np.hstack((ts_h, ts_k))
# Определим температуры воды за подогревателями с учетом недогрева 3С
tw = ts_h - 3
# Разобьем массив на части по нагреву основного конденсата и пит.воды
tw_pvd = tw[0:3]
tw_pnd = tw[3:]
# Зададим давление питтаельной воды и основного конденсата
Ppv = 1.3 * P0 / 10 ** 6
Pok = 1
# Создадим массив давлений
Pw = np.array([Ppv, Ppv, Ppv, Pok, Pok, Pok, Pok, Pok, Pok])
# Найдем энтальпии воды за подогревателями
# за ПВД
hw_pvd = np.round(PropsSI('H', 'P', Ppv * 10 ** 6,'T', tw_pvd+273.15, 'IF97::Water') / 10**3, 3)
# за ПНД
hw_pnd = np.round(PropsSI('H', 'P', Pok * 10 ** 6,'T', tw_pnd+273.15, 'IF97::Water') / 10**3, 3)
# Склеим массивы
hw = np.hstack((hw_pvd, hw_pnd, hdr_k))
tw = np.hstack((tw_pvd, tw_pnd))
tw1 = np.hstack((tw, ts_k))
# Определим перепад энтальпий пара между входом в ЦНД и выходом ЦВД
dhpp = hcnd0 - hcvd_i[3] 
# Определим удельную работу пара в отборе
Hj_cvd = h0 - hcvd_i
Hj_cnd = h0 - hcnd_i + dhpp 
Hj = np.hstack((Hj_cvd, Hj_cnd))
yj = (Hi - Hj) / Hi
# Определим температуру пара в отборах
t_id =  np.round(PropsSI('T', 'P', Potb * 10 **6,'H', hi_d * 1000, 'IF97::Water') -273.15, 1)
# Определим энтальпию дренажей
hdr = np.round(PropsSI('H', 'P', Potb * 10 **6, 'Q', 0, 'IF97::Water') / 10**3, 3)
# Делаем общую сводную таблицу
s_list = ['П1','П2','П3', 'П4', 'П5', 'П6', 'П7', 'П8', 'К' ]
data = {'Точка отб.' : s_list,
    'Pотб, МПа' : Potb,
    'tотб, С' : t_id,
    'hi, кДж/кг' : hi_d,
    'Pп, МПа ' : Pheater1,
     'ts, C' : ts_h1,
    'hдр, кДж/кг' : hdr,
       'tпв, С' : tw1,
       'Pпв, МПа' : Pw,
      'hпв, кДж/кг' : hw,
       'Hj, кДж/кг' :Hj,
       'Yj': yj}

df_new = pd.DataFrame(data).set_index('Точка отб.')
#df_new.to_csv('100.csv', index=False)
df_new

,"Pотб, МПа","tотб, С","hi, кДж/кг","Pп, МПа","ts, C","hдр, кДж/кг","tпв, С","Pпв, МПа","hпв, кДж/кг","Hj, кДж/кг",Yj
Точка отб.,,,,,,,,,,,
П1,2.786,395.8,3225.6,2.64670,227.0,989.220,224.0,5.72,962.982,99.844,0.904828
П2,1.930,352.5,3144.6,1.83350,208.0,900.422,205.0,5.72,876.514,180.844,0.827619
П3,1.288,308.6,3062.8,1.22360,188.8,812.862,185.8,5.72,791.085,262.644,0.749647
П4,0.508,221.5,2901.3,0.48260,150.5,642.775,147.5,1.00,621.812,424.144,0.595704
П5,0.300,189.1,2843.4,0.28500,131.8,561.455,128.8,1.00,541.768,509.593,0.514254
П6,0.127,115.4,2704.4,0.12065,104.9,446.249,101.9,1.00,427.784,648.593,0.381758
П7,0.058,85.1,2593.9,0.05510,83.8,356.191,80.8,1.00,339.062,759.093,0.276429
П8,0.029,68.3,2504.1,0.02755,67.2,285.974,64.2,1.00,269.541,848.893,0.190832
К,0.005,32.9,2303.9,0.00500,32.9,137.765,32.9,1.00,137.765,1049.093,0.000000


In [87]:
hcnd0

2928.849

## Для составления системы уравнений запишем все необходимые данные по энтальпиям отборов, основного конденсата и питательной воды, дренажей (ПВД, ПНД, СПП)

### Данные энтальпий в отборах пара

In [19]:
h1 = hi_d[0]
h2 = hi_d[1]
h3 = hi_d[2]
h4 = hi_d[3]
h5 = hi_d[4]
h6 = hi_d[5]
h7 = hi_d[6]
h8 = hi_d[7]
hk = hi_d[8]

### Данные энтальпий воды за подогревателями и конденсатором

In [7]:
type(h3)

float

In [21]:
Pd = 0.6 # Давление в деаэраторе
# Определим энтальпию воды после деаэратора
hwd = np.round(PropsSI('H', 'P', Pd * 10 ** 6, 'Q', 0, 'IF97::Water') / 10**3, 1)
# Определим энтальпию выпара в деаэраторе 
hpd = np.round(PropsSI('H', 'P', Pd * 10 ** 6, 'Q', 1, 'IF97::Water') / 10**3, 1)
# Относительный расход пара на эжектора
aeg = 0.005
# Энтальпии воды за подогревателями
hw1 = hw[0]
hw2 = hw[1]
hw3 = hw[2]
hw4 = hw[3]
hw5 = hw[4]
hw6 = hw[5]
hw7 = hw[6]
hw8 = hw[7]
hwk = np.round((hw[8] + (2 + 4) * 4.2), 1) # Энтальпия воды после нагрева в ОЭ и ОУ
hpn = 676.3

In [9]:
type(hpd)
hpd

2756.1

### Данные энтальпий дренажей

In [22]:
hdr1 = hdr[0]
hdr2 = hdr[1]
hdr3 = hdr[2]
hdr4 = hdr[3]
hdr5 = hdr[4]
hdr6 = hdr[5]
hdr7 = hdr[6]
hdr8 = hdr[7]

### Данные по тракту сепаратора пароперегревателя

In [46]:
# Давление на выходе из сепаратора
Psep = 0.98 * P4
# Степень сухости на выходе из ЦВД 
Xcvd = np.round(PropsSI('Q', 'P', Psep *10 ** 6, 'H', h4 * 1000, 'IF97::Water'), 3)
# Степень сухости пара на выходе из сепаратора
Xsep = 0.995
# Энтальпия пара на выходе из сепаратора
hsep = np.round(PropsSI('H', 'P', Psep * 10 ** 6, 'Q', Xsep, 'IF97::Water') / 10**3, 1)
# Энтальпия дренажа сепаратора
hsep_dr = float(np.round(PropsSI('H', 'P', Psep * 10 ** 6, 'Q', 0, 'IF97::Water') / 10**3, 1))
# Температура пара на выходе из ПП1
tpp1 = t_id[0] - 15
# Давление на выходе из ПП1
Ppp1 = 0.98 * Psep
# Энтальпия пара на выходе из ПП1
hpp1 = np.round(PropsSI('H', 'P', Ppp1 * 10 ** 6, 'T', tpp1 + 273.15, 'IF97::Water') / 10**3, 1)
hpp_dr = hdr1
hpp_dr
# Энтальпия пара на выходе из ПП2
hpp2 = float(hcnd0)
# Энтальпия дренажа ПП2
hpp2_dr = np.round(PropsSI('H', 'P', P0, 'Q', 0, 'IF97::Water') / 10**3, 1)
# Удал влага
yvlag = (Xsep - Xcvd)/Xsep
hpp2_dr

1108.6

In [43]:
Xcvd

0.885

## Составление и решение системы уравнений.
#### Решать уравнение будем, используя модуль sympy для символьного исчисления
#### `Сначала создадим символы всех неизвестных`

In [11]:
# Относительный расход питательной воды
#apv = sym.Symbol('apv')
# Относительный расход отсепарированной влаги
asep = sym.Symbol('asep')
# Относительный расход пара на ПП1
app1 = sym.Symbol('app1')
# Относительный расход пара на ПП2
app2 = sym.Symbol('app2')
# Относительные расходы пара на ПВД и Деаэаратор
a1 = sym.Symbol('a1')
a2 = sym.Symbol('a2')
a3 = sym.Symbol('a3')
ad = sym.Symbol('ad')
# Относительный расход основного конденсата
aok = sym.Symbol('aok')
# Относительные расходы пара на ПНД
a4 = sym.Symbol('a4')
a5 = sym.Symbol('a5')
a6 = sym.Symbol('a6')
a7 = sym.Symbol('a7')
a8 = sym.Symbol('a8')
# Относительный расход на выходе из конденсатора
aok1 = sym.Symbol('aok1')
# Точка смешения 2- горячая , 1 -холодная
hsm1 = sym.Symbol('hsm1')
hsm2 = sym.Symbol('hsm2')



### Составляем систему уравнений

In [58]:
hpp2_dr 

1108.6

In [67]:
sym.solve((asep - yvlag * (1 - a1 - a2 - a3  - app1 - ad -a4), 
         (1 - a1 - a2 - a3 - a4 - app1 - ad) * h4 - asep * hsep_dr - (1 - a1 - a2 - a3 - a4 - app1 - ad - asep) * hsep,
         app1 * (h1 - hdr1) - (1 - a1 - a2  - a3 - a4 - app1 - ad - asep) * (hpp1 - hsep),
         app2 * (h0 - hpp2_dr) - (1 - a1 - a2 - a3 - a4 - app1 - ad - asep) * (hpp2 - hpp1),
         apv - 1.015 - app2,
         a1 * (h1 - hdr1) + app2 * (hpp2_dr - hdr1) - apv *(hw1 -hw2),
        (a1 + app2) * (hdr1 - hdr2) + a2 * (h2 - hdr2) - apv *(hw2 -hw3),
        (a1 + app2 + a2) * (hdr2 - hdr3) + a3 * (h3 - hdr3) + app1 * (hdr1 - hdr3) - apv *(hw3 -hpn),
        (aok1 + a6 + a7 + a5 + a4) * hw4 + (a1 + a2 + a3 + app1 + app2) * hdr3 + ad * h3 + asep * hsep_dr - apv * hwd - aeg * hpd,
        aok1 + a6 + a7 + a5 + a4 + a1 + a2 + a3 + app1 + app2 + ad  + asep  - apv - aeg,
        a4 * (h4 - hdr4) - (aok1 + a6 + a7 + a5 + a4) * (hw4 - hsm2),
        (a4 + a5) * hdr5 + (aok1 + a6 + a7) * hw5 - (aok1 + a6 + a7 + a5 + a4) * hsm2,
        a5 * (h5 - hdr5) + a4 * (hdr4 - hdr5) - (aok1 + a6 + a7) * (hw5 - hw6),
        a6 * (h6 - hdr6) - (aok1 + a6 + a7) * (hw6 - hsm1),
        (a6 + a7) * hdr7 + (aok1) * hw7 - (aok1 + a6 + a7) * hsm1,
        a7 * (h7 - hdr7) + a6 * (hdr6 - hdr7) - aok1 * (hw7 - hw8),
        a8 * (h8 - hdr8) - aok1 * (hw8 - hwk)), apv, asep, app1, app2, a1, a2, a3, ad, a4, a5, a6, a7, a8, aok1, hsm1, hsm2)

[]

In [68]:
apv - 1.015 - app2

-app2 + apv - 1.015

In [ ]:
sym.solve([a1 * (h1 - hdr1) - apv *(hw1 -hw2),
        (a1) * (hdr1 - hdr2) + a2 * (h2 - hdr2) - apv *(hw2 -500),
        (a1 + a2) * hdr2 + aok * hw4 - apv * hsm1,
        a1 + a2 + aok - apv,
        a4 * (h4 - hdr4) - aok * (hw4 - 300)], 

In [26]:
hpp2_dr

1108.6

In [65]:
f1 = sym.expand(asep - yvlag * (1 - a1 - a2 - a3 - a4 - app1 - ad))
f2 = sym.expand((1 - a1 - a2 - a3 - a4 - app1 - ad) * h4 - asep * hsep_dr - (1 - a1 - a2 - a3 - a4 - app1 - ad - asep) * hsep)
f3 = sym.expand(app1 * (h1 - hdr1) - (1 - a1 - a2  - a3 - a4 - app1 - ad - asep) * (hpp1 - hsep))
f4 = sym.expand(app2 * (h0 - hpp2_dr) - (1 - a1 - a2 - a3 - a4 - app1 - ad - asep) * (hpp2 - hpp1))
f5 = sym.expand(apv - 1.015 - app2)
f6 = sym.expand(a1 * (h1 - hdr1) + app2 * (hpp2_dr - hdr1) - apv *(hw1 -hw2))
f7 = sym.expand((a1 + app2) * (hdr1 - hdr2) + a2 * (h2 - hdr2) - apv *(hw2 -hw3))
f8 = sym.expand((a1 + app2 + a2) * (hdr2 - hdr3) + a3 * (h3 - hdr3) + app1 * (hdr1 - hdr3) - apv *(hw3 -hpn))
f9 = sym.expand(aok * hw4 + (a1 + a2 + a3 + app1 + app2) * hdr3 + ad * h3 + asep * hsep_dr - apv * hwd - aeg * hpd)
f10 = sym.expand(aok + a1 + a2 + a3 + app1 + app2 + ad  + asep  - apv - aeg)
f11 = sym.expand(a4 * (h4 - hdr4) - aok * (hw4 - hsm2))
f12 = sym.expand((a4 + a5) * hdr5 + (aok1 + a8 + a6 + a7) * hw5 - aok * hsm2)
f13 = sym.expand(aok - (aok1 + a8 + a6 + a7 + a5 + a4))
f14 = sym.expand(a5 * (h5 - hdr5) + a4 * (hdr4 - hdr5) - (aok1 + a8 + a6 + a7) * (hw5 - hw6))
f15 = sym.expand(a6 * (h6 - hdr6) - (aok1 + a8 + a6 + a7) * (hw6 - hsm1))
f16 = sym.expand((a5 + a6) * hdr7 + (aok1 + a8) * hw7 - (aok1 + a8 + a6 + a7) * hsm1)
f17 = sym.expand(a7 * (h7 - hdr7) + a6 * (hdr6 - hdr7) - (aok1 + a8) * (hw7 - hw8))
f18 = sym.expand(a8 * (h8 - hdr8) - (aok1 + a8) * (hw8 - hwk))

In [35]:
lin_s = [f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f14, f15, f16, f17]

In [64]:
mnk_list = [hsep_dr, hsep, h1 - hdr1, hpp1 - hsep, h0 - hpp2_dr, hpp2 - hpp1, h1 - hdr1,
hpp2_dr - hdr1, hw1 -hw2, hdr1 - hdr2, h2 - hdr2, hw2 -hw3, hdr2 - hdr3,
h3 - hdr3, hdr1 - hdr3, hw3 -hpn, hdr3, h3, hsep_dr, hwd, hpd,
h4 - hdr4, hdr5, hw5, h5 - hdr5, hdr4 - hdr5, hw5 - hw6, h6 - hdr6, hdr7,
hw7, h7 - hdr7, hdr6 - hdr7, hw7 - hw8, h8 - hdr8, hw8 - hwk]

for i in mnk_list:
    print(i, type(i))

639.5 <class 'float'>
2737.4 <class 'numpy.float64'>
1751.18 <class 'numpy.float64'>
150.5999999999999 <class 'numpy.float64'>
1690.67 <class 'numpy.float64'>
42.30699999999979 <class 'numpy.float64'>
1751.18 <class 'numpy.float64'>
119.37999999999988 <class 'numpy.float64'>
86.48399999999992 <class 'numpy.float64'>
88.798 <class 'numpy.float64'>
1785.478 <class 'numpy.float64'>
85.44200000000001 <class 'numpy.float64'>
87.56000000000006 <class 'numpy.float64'>
1815.438 <class 'numpy.float64'>
176.35800000000006 <class 'numpy.float64'>
114.7220000000001 <class 'numpy.float64'>
812.862 <class 'numpy.float64'>
2628.3 <class 'numpy.float64'>
639.5 <class 'float'>
670.5 <class 'numpy.float64'>
2756.1 <class 'numpy.float64'>
1862.4249999999997 <class 'numpy.float64'>
561.455 <class 'numpy.float64'>
541.768 <class 'numpy.float64'>
2282.945 <class 'numpy.float64'>
81.31999999999994 <class 'numpy.float64'>
113.98400000000004 <class 'numpy.float64'>
2259.0510000000004 <class 'numpy.float64'>
35

In [51]:
type(hsep_dr)

float

In [21]:
h1 = 3034.4
h2 = 2941.7
#h3 = 3303.1
h4 = 3191.6
h5 = 3015
h6 = 2929.9
h7 = 2706.1
h8 = 2602.3
hk = 2367.3

In [23]:
hw1 = 1205.5
hw2 = 1063.1
#hw3 = 
hw4 = 746.4
hw5 = 624.9
hw6 = 510
hw7 = 393
hw8 = 214.2
hwk = 135

In [24]:
hdr1 = 1224.2
hdr2 = 1073.5
#hdr3 = 842.7
hdr4 = 758.8
hdr5 = 632.3
hdr6 = 517
hdr7 = 393
hdr8 = 221.7
apv = 1.05
dh = 32.7

In [27]:
# Ольгино задать apv
sym.solve([a1 * (h1 - hdr1) - apv *(hw1 -hw2),
        a1 * (hdr1 - hdr2) + a2 * (h2 - hdr2) - apv *(hw2 -hsm1-32.7),
        (a1 + a2) * hdr2 + aok * hw4 - apv * hsm1,
        a1 + a2 + aok - apv,
        aok - (aok1 + a4 + a5 + a6 + a7),
        a4 * (h4 - hdr4) - aok * (hw4 - hw5), 
        a5 * (h5 - hdr5) + a4 * (hdr4 - hdr5) - aok * (hw5 - hw6),
        a6 * (h6 - hdr6)  + (a4 + a5) * (hdr5 - hdr6) - aok * (hw6 - hw7),
        a7 * h7 + aok1 * hw8 + (a4 + a5 + a6) * hdr6 - aok * hw7,
        a8 * (h8 - hdr8) - aok1 *(hw8 - hwk)], a1, a2, a4, a5, a6, a7, a8, aok1, hsm1, aok, dict=True)

[{a1: 0.0825986078886311,
  a2: 0.117858326948851,
  a4: 0.0424282647226430,
  a5: 0.0387146190035502,
  a6: 0.0373164093540489,
  a7: 0.0465623925945507,
  a8: 0.0227732896141426,
  aok1: 0.684521379487725,
  hsm1: 808.847107986038,
  aok: 0.849543065162518}]